# Obtaining the genomic length of each TE

For the purposes of normalizing the data into RPKM (reads per kilobase of transcript, per million mapped reads), I need to know the total length of each TE in the genome.

I iterated over the TE annotation gtf, which includes the length and gene/class/family id for each TE, and added together the lengths of each copy of the same TE.

I also determined the total lengths of each class and family of TEs by adding together every copy of every TE for each class and family.

Lastly, I also included how many copies of each were encountered, in case it would be useful for later.

In [29]:
import re

import pandas as pd

tepath = "/global/projectb/scratch/cmodonog/TransposableElements"
te_lengths = dict()

i=0
with open("%s/Sbicolor_reference/Sbicolor_313_v3.1.repeatmasked_assembly_v3.0.gtf" % tepath,"r") as inf_te_anno:
    for line in inf_te_anno:
        i+=1
        linedata = line.split()
        length = int(linedata[4]) - int(linedata[3])
        gene_id = "gene:" + re.sub(r'[";]+', "", linedata[9])
        family_id = "family:" + re.sub(r'[";]+', "", linedata[13])
        # some 'classes' are just the gene name repeated. Don't count these as true classes.
        if linedata[15] != linedata[9]:
            class_id = "class:" + re.sub(r'[";]+', "", linedata[15])
        ids = [gene_id, family_id, class_id]
        for name in ids:
            if name not in te_lengths:
                # Want to keep a count of both total base length and number of copies in the genome
                te_lengths[name] = [length, 1]
            else:
                te_lengths[name][0] += length
                te_lengths[name][1] += 1
df_lengths = pd.DataFrame.from_dict(te_lengths, orient='index', 
                                  columns = ['Total length', 'Copies in genome'])
df_lengths = df_lengths.reset_index().rename(columns={'index':'Name'})
series_lengths = pd.Series(df_lengths['Total length'].values, index=df_lengths['Name'])
print(df_lengths)
print(series_lengths)
df_lengths.to_csv('te_counts.txt', index=False, sep='\t')

                          Name  Total length  Copies in genome
0             gene:EnSpm-N1_SB        618877              1390
1                   family:DNA      68548970            168148
2              class:CMC-EnSpm      36502497             46732
3               gene:Os4_05_6L         31226               137
4                   family:LTR     365711476            199920
5             gene:hAT-N18_SBi        868145              4381
6             class:hAT-Tip100       2038453              6998
7           gene:EnSpm-N13_SBi        154525               343
8                gene:Zm9L_69L         96161               444
9           gene:EnSpm-N12_SBi        247388               734
10                  gene:CLOUD        287457               765
11                  class:MuDR       1156552              2878
12               gene:L1-7_SBi        177353               260
13                 family:LINE      10037487             10342
14                    class:L1      10309916           

## Reloading the TE annotation count/length table 

The below cell is to reload df_lengths and series_lengths from the text file created in the last cell.

For using the data again after refreshing the jupyter notebook.

In [22]:
import pandas as pd

df_lengths = pd.read_csv('te_counts.txt', sep='\t')
series_lengths = pd.read_csv('te_counts.txt', sep='\t', usecols=[0,1], index_col=0, squeeze=True)
print(series_lengths)
#print(df_lengths)

%timeit length = df_lengths.loc[df_lengths['Name'] == "gene:SRSiOTOT00000016"].iloc[0]["Total length"]
%timeit length = series_lengths["gene:SRSiOTOT00000016"]

Name
gene:EnSpm-N1_SB               618877
family:DNA                   68548970
class:CMC-EnSpm              36502497
gene:Os4_05_6L                  31226
family:LTR                  365711476
gene:hAT-N18_SBi               868145
class:hAT-Tip100              2038453
gene:EnSpm-N13_SBi             154525
gene:Zm9L_69L                   96161
gene:EnSpm-N12_SBi             247388
gene:CLOUD                     287457
class:MuDR                    1156552
gene:L1-7_SBi                  177353
family:LINE                  10037487
class:L1                     10309916
gene:Copia-119_SB-LTR            1635
class:Copia                  51261186
gene:Copia-71_SB-LTR             9562
gene:LTR-1_SBi                2033359
class:Gypsy                 330480728
gene:Gypsy-16_SB-LTR          1140381
gene:Copia-71_SB-I              40118
gene:LINE1-40_SBi               36501
gene:EnSpm-N11_SBi            1087737
gene:Gypsy5-SB_LTR            5294317
gene:EnSpm3_SB                 281301
gene:En

The whole dataframe contains all of the data, but for iterating I turn the first column into a series, as looking up values in a series is much faster.

# Parsing through the TEcount output for each library

In [26]:
import os
import sys
import re
from datetime import datetime

import numpy as np
import pandas as pd

def cnt_table_parse(file, library_name, df_md, df_lib):
    total_expcount = 0
    columns = ['Sample name', 'Library name', 'Week', 'Genotype', 
             'Tissue', 'Treatment', 'Replicate']
    # Sample name is the fifth column in this table.
    sample_name = df_lib.loc[df_lib["1=libraryName"] == library_name].iloc[0][4]
    # Date is the sixth column in this table.
    week = df_md.loc[df_md["Sample name"] == sample_name].iloc[0][7]
    genotype = df_md.loc[df_md["Sample name"] == sample_name].iloc[0][9]
    tissue = df_md.loc[df_md["Sample name"] == sample_name].iloc[0][3]
    treatment = df_md.loc[df_md["Sample name"] == sample_name].iloc[0][10]
    treatment = int(treatment[-1])
    replicate = df_md.loc[df_md["Sample name"] == sample_name].iloc[0][11]
    replicate = int(replicate)
    uniq_expcount = [sample_name, library_name, week, genotype, 
                     tissue, treatment, replicate]
    
    counting_dict = dict()
    semi_useless_first_line = file.readline()
    # now that that's out of the way...
    for line in file:
        count = int(line.split()[1])
        name = line.split()[0]
        total_expcount += count
        if "Sobic" in name:
            continue
        names = line.split()[0].split(':')
        # If gene and class have the exact same name, then the 'class' only contains that gene
        # In this case, don't include both class;fam and gene;cls;fam as they would be identical
        if names[0] != names[2]:
            names = ["gene:%s;cls:%s;fam:%s" % (names[0], names[2], names[1]),
                     "class:%s;fam:%s" % (names[2], names[1]),
                     "family:%s" % names[1]]
        else:
            names = ["gene:%s;cls:%s;fam:%s" % (names[0], names[2], names[1]),
                     "family:%s" % names[1]]
        for name in names:
            if name not in counting_dict:
                counting_dict[name] = count
            else:
                counting_dict[name] += count
    for name in sorted(counting_dict):
        te_kb = series_lengths[name.split(';')[0]]
        rpkm_count = counting_dict[name]*10e9/(total_expcount * te_kb)
        columns.append(name)
        uniq_expcount.append(rpkm_count)
    return columns, uniq_expcount



def main():
    tepath = "/global/projectb/scratch/cmodonog/TransposableElements"
    df_md = pd.read_excel('%s/EPICON_samples_reference.xlsx' % tepath)
    df_lib = pd.read_table('%s/LIBRARIES_TABLE.txt' % tepath)
    count = 0
    entries = dict()
    print("Processing libraries...")
    for dir, subdirs, files in os.walk("%s/Sbicolor_libraries" % tepath):
        for file in files:
            if "cntTable" in file:
                cnt_table_path = "%s/%s" % (dir, file)
                library_name = dir.split('/')[-1]
                with open(cnt_table_path) as cnt_table:
                    columns, uniq_expcount = cnt_table_parse(cnt_table, library_name,
                                                           df_md, df_lib)
                entries[library_name] = uniq_expcount
                count += 1
                if count % 50 == 0:
                    print("%d libraries processed..." % count)
    dataframe = pd.DataFrame.from_dict(entries, orient='index',
                                       columns=columns)
    df_TEcount_summary = dataframe.sort_values(by=['Genotype', 'Tissue', 'Treatment', 'Week', 'Replicate'])
    print("Done processing libraries!")
    print("Writing excel file...")
    excelwriter = pd.ExcelWriter('EPICON_countdata.xlsx')
    df_TEcount_summary.to_excel(excelwriter, 'Sheet1')
    excelwriter.save()
    print("Done!")
    return dataframe

if __name__ == "__main__":
    df_TEcount_summary = main()


Processing libraries...
50 libraries processed...
100 libraries processed...
150 libraries processed...
200 libraries processed...
250 libraries processed...
300 libraries processed...
350 libraries processed...
Done processing libraries!
Writing excel file...
Done!


## Reloading the TEcount summary table

The below cell is to reload df_TEcount_summary from the text file created in the last cell.

For using the data again after refreshing the jupyter notebook.

In [33]:
import pandas as pd

df_TEcount_summary = pd.read_excel("EPICON_countdata.xlsx")
df_TEcount_summary

,Sample name,Library name,Week,Genotype,Tissue,Treatment,Replicate,class:CMC-EnSpm;fam:DNA,class:Caulimovirus;fam:LTR,class:Copia;fam:LTR,...,gene:victim_334B07-1;cls:Copia;fam:LTR,gene:waba_134O12-1hdnm;cls:waba_134O12-1hdnm;fam:Other,gene:wiwa_af391808-0;cls:Copia;fam:LTR,gene:xilon_af448416-1;cls:Gypsy;fam:LTR,gene:xilon_af448416-2hdnm;cls:Gypsy;fam:LTR,gene:yemi_013I05-1hdnm;cls:yemi_013I05-1hdnm;fam:LTR,gene:yote_438D03-1;cls:yote_438D03-1;fam:DNA,gene:yote_af090447-1;cls:yote_af090447-1;fam:DNA,gene:zeon_ac148394-1hdnm;cls:Gypsy;fam:LTR,gene:zmgypsy_part;cls:Gypsy;fam:LTR
BOCAZ,0622162L06,BOCAZ,3,BTx642,L,1,1,8.028959,0.370586,2.542128,...,0.0,0.323955,0.000000,0,0,5.289925,10.633384,0.000000,0,0.000000
BOCHA,0622162L14,BOCHA,3,BTx642,L,1,2,6.666978,0.000000,2.596219,...,0.0,0.499150,0.000000,0,0,14.942982,9.060398,0.000000,0,0.000000
BOPGS,0622162L17,BOPGS,3,BTx642,L,1,3,7.064250,0.568902,2.717656,...,0.0,0.462153,0.000000,0,0,2.706929,7.492871,0.000000,0,0.000000
BOCBP,0629164L06,BOCBP,4,BTx642,L,1,1,7.807358,0.000000,2.675941,...,0.0,0.415813,0.000000,0,0,9.145583,15.972739,0.000000,0,0.000000
BOCHS,0629164L14,BOCHS,4,BTx642,L,1,2,7.251272,0.917672,3.175676,...,0.0,0.544524,0.000000,0,0,23.578730,8.978492,0.000000,0,1.426913
BOPHB,0629164L17,BOPHB,4,BTx642,L,1,3,6.859350,0.556697,2.600733,...,0.0,0.314600,0.000000,0,0,13.244268,10.823601,0.000000,0,0.000000
BOCCA,0706165L06,BOCCA,5,BTx642,L,1,1,6.760803,0.436495,2.334619,...,0.0,0.289069,0.000000,0,0,9.346112,20.942669,0.000000,0,0.000000
BOCCS,0713166L06,BOCCS,6,BTx642,L,1,1,6.404230,0.419383,2.836591,...,0.0,0.299955,0.000000,0,0,23.945920,17.951679,0.000000,0,0.000000
BOCNB,0713166L14,BOCNB,6,BTx642,L,1,2,6.530560,0.990315,3.310443,...,0.0,0.356775,0.000000,0,0,22.617984,21.800774,0.000000,0,0.000000
BOPHT,0713166L17,BOPHT,6,BTx642,L,1,3,6.447026,0.211955,3.696893,...,0.0,0.314422,0.000000,0,0,9.076641,19.142439,0.000000,0,0.000000


# Identifying interesting TEs

In supplement to the heatmaps I'll be producing with Morpheus, I want to do some simple statistical analysis to find a handful of TEs that seem to have significantly
different expression during either of the two treatments compared to the control, so that I know the best TEs to make some graphs out of in JMP.

While there are 353 libraries, there are only 12 total combinations of conditions to look at individually.
genotype: RTx430 vs BTx642
tissue: Leaf vs Root
Treatment: 1, 2, or 3

Within each of those combinations, we want to look at expression over the course of time in weeks. The replicates for each week are averaged to get the estimated expression value.

While a true statistical analysis would likely be more sophisticated, I do the following:

1. separate the big pandas dataframe into 12, for each of the combinations listed above.
2. For each TE in each dataframe, calculate the spearman correlation using weeks as the x and counts as the y.
3. Compare spearman correlations between treatments: simply the difference between each of the two treatments (Irr 1 and Irr 2) and the control
4. Select TEs based off of maximum difference in spearman correlation in any condition/treatment

In [150]:
import numpy as np
import pandas as pd
import scipy.stats

def compare_to_control(array):
    treat1 = array[0] - array[2]
    treat2 = array[1] - array[2]
    return treat1, treat2

def findmax(nested_array):
    nparr = np.array(nested_array)
    @
    rho_values = nparr[:, 1].astype(np.float).reshape((4,3))
    rho_compare = np.apply_along_axis(compare_to_control, 1, rho_values).flatten()
    return np.max(np.absolute(rho_compare))

all_TEs = list(df_TEcount_summary.columns.values[7:])
# I'm not going to do statistical analysis on the samples with 0 expression
relevant_TEs = []
for TE in all_TEs:
    TE_count_values = np.array(df_TEcount_summary[TE].values)
    if np.mean(TE_count_values) < 10:
        continue
    if not TE.startswith("gene:"):
        continue
    relevant_TEs.append(TE)
print(len(relevant_TEs))

sp_corrs = {}
for genotype in ["RTx430", "BTx642"]:
    for tissue in ["L", "R"]:
        for treatment in [1, 2, 3]:
            print("Processing TEs for Genotype %s Tissue %s Treatment %d" % (genotype, tissue, treatment))
            df = df_TEcount_summary.loc[(df_TEcount_summary["Genotype"] == genotype) & (df_TEcount_summary["Tissue"] == tissue) & (df_TEcount_summary["Treatment"] == treatment)]
            date_values = df["Week"].values
            for TE in relevant_TEs:
                TE_counts = df[TE].values
                rho, pval = scipy.stats.spearmanr(date_values, TE_counts)
                if TE not in sp_corrs:
                    sp_corrs[TE] = [["Geno:%s;Tissue:%s;Irr:%d" % (genotype, tissue, treatment), rho, pval]]
                else:
                    sp_corrs[TE].append(["Geno:%s;Tissue:%s;Irr:%d" % (genotype, tissue, treatment), rho, pval])

sorted_sp_corrs = sorted(sp_corrs.items(), key=lambda x: findmax(x[1]))
top_results = sorted_sp_corrs[:10]
for item in top_results:
    print(item[0])

            

587
Processing TEs for Genotype RTx430 Tissue L Treatment 1
Processing TEs for Genotype RTx430 Tissue L Treatment 2
Processing TEs for Genotype RTx430 Tissue L Treatment 3
Processing TEs for Genotype RTx430 Tissue R Treatment 1
Processing TEs for Genotype RTx430 Tissue R Treatment 2
Processing TEs for Genotype RTx430 Tissue R Treatment 3
Processing TEs for Genotype BTx642 Tissue L Treatment 1
Processing TEs for Genotype BTx642 Tissue L Treatment 2
Processing TEs for Genotype BTx642 Tissue L Treatment 3
Processing TEs for Genotype BTx642 Tissue R Treatment 1
Processing TEs for Genotype BTx642 Tissue R Treatment 2
Processing TEs for Genotype BTx642 Tissue R Treatment 3
gene:CACTA-I;cls:En-Spm;fam:DNA
gene:CLOUD-7;cls:MuDR;fam:DNA
gene:CLOUD-4;cls:MuDR;fam:DNA
gene:Copia-16_SB-LTR;cls:Copia;fam:LTR
gene:Copia-132_SB-LTR;cls:Copia;fam:LTR
gene:ARSiTRTA00000001;cls:Simple;fam:Other
gene:Copia-112_SB-LTR;cls:Copia;fam:LTR
gene:Copia-33_SB-LTR;cls:Copia;fam:LTR
gene:Copia-11_SB-LTR;cls:Copia;